## Лабораторная работа № 5 "Расчет коэффициента размножения нейтронов в гомогенном реакторе с использованием формулы четырех сомножителей"

Работу выполнил: <ФИО, группа>

In [51]:
import pandas as pd
import numpy as np
from typing import Dict
from itertools import product
from matplotlib.lines import Line2D
from matplotlib import pyplot as plt

## Данные
В качестве исходных данных в работе используются
- обогащение урана $x$ - отношение доли U-235 к доле U-238 в урановом топливе, $x = \dfrac{N_{235}}{N_{238}}$  
- $\sigma_f, \sigma _c, \sigma _s$ - микросечения деления (f), поглощения (c), рассеяния (s), $10^{-24} см^2 = 1\ б$  
- $\rho$ - плотность вещества, $[г/см^3]$  
- $M$ - молярная масса, $[г/моль]$
- $\xi$ - средний логарифмический декремент энергии
- $\epsilon$ - объемная доля компоненты гомогенного реактора  


Обогащение: $x \in [0.00714, 0.01, 0.02, 0.03, 0.04, 0.05]$

Типы компонент:  
- топливо [T] ($U, UO_2$)  
- теплоноситель [ТН] ($H_2O, D_2O$)  
- замедлитель [З] ($H_2O, D_2O, C$)  
- конструкционные материалы [К] ($Zr, Fe$)  
В качестве значений молярной массы  используйте молярные массы **наиболее распространенных в природе** изотопов.  
Необходимо выяснить распространенность и извлечь значения из библиотеки `mendeleev`, явным образом отразить поиск и выбор в тексте работы.

Ниже приведена заготовка для таблицы параметров. Все параметры, необходимые для расчетов и непосредственно характеризующие свойства вещества, должны быть внесены в эту таблицу по ходу работы.  
Содержимое колонки `type` для удобства можно преобразовать.

In [69]:
params = pd.DataFrame(data={'rho':  [1.00, 1.10, 1.60, 6.51, 7.87], # плотность вещества, г/см^3
                            's_c':  [0.66, 0.001, np.nan, np.nan, np.nan], # микросечение захвата, б (необходимо дополнить)
                            's_s':  [63, 13.6, np.nan, np.nan, np.nan], # микросечение рассеяни, б  (необходимо дополнить)
                            'decr': [0.948, 0.57, 0.158, 0.022, 0.035], # лог. декремент энергии, б/р
                            'type': ['З/ТН', 'З/ТН', 'З', 'К', 'К']}, # тип компоненты реактора
                             index= ['H2O','D2O', 'C', 'Zr', 'Fe'])  # название вещества
params

,rho,s_c,s_s,decr,type
H2O,1.00,0.660,63.0,0.948,З/ТН
D2O,1.10,0.001,13.6,0.570,З/ТН
C,1.60,NaN,NaN,0.158,З
Zr,6.51,NaN,NaN,0.022,К
Fe,7.87,NaN,NaN,0.035,К


In [71]:
# пример использования на будущее
C_pars = params.loc['C', ['rho', 'type']]
C_pars, type(C_pars)

(rho     1.6
 type      З
 Name: C, dtype: object,
 pandas.core.series.Series)

Формат данных для топлива отличается от формата данных для других компонент, поэтому будем записывать отдельно

In [65]:
fuel_rho = {'U': 18.9, 'UO2': 10.96}
fuel_params = pd.DataFrame(data={}, index = ['U_235', 'U_238'])

In [66]:
fuel_params

""
U_235
U_238


Дополните таблицы `params` и `fuel_params` значениями микросечений деления, рассеяния и захвата по данным [РОССФОНД-2010](https://www-nds.iaea.org/exfor/endf.htm) при тепловых энергиях (0.025 эВ). Сечение деления указывать только для U-235, для U-238 - 0.

Напоминание по поиску: в зависимости от типа реакции, в поле Reaction необходимо ввести:  
- *N,F* (деление),  
- *N,EL* (рассеяние), 
- *N,G* (захват).


In [72]:
# дополните таблицы params и fuel_params микросечениями


В качестве долей компонент реактора в работе используются доли материалов в реальных ячейках реакторов типа
1. РБМК: Т - 0.0316, ТН - 0.038, К - 0.903, З - 0.0272.
2. ВВЭР: Т - 0.002, ТН - 0.9974, К - 0.002, З - 0.0. 

Обратите внимание, в случае ВВЭР теплоносителем и замедлителем является одно и то же вещество. Тогда в качестве свойств (например, $\sigma, \xi$) замеделителя надо брать свойства теплоносителя. В расчетах, в которых используется (не)взвешанное суммирование объемных концентраций $N$ теплоносителя и замедлителя, использовать соответствующее значение **единожды**. 

In [16]:
# отразите в программном виде доли компонент

## Формулы расчета  
Формула четырех сомножителей: $k_{\infty} = \dfrac{n'}{n} =\dfrac{n\ \mu \phi \theta \eta}{n} = \mu \phi \theta \eta$, где  
$\mu$ - коэффициент размножения на быстрых нейтронах;  
$\phi$ - вероятность избежать резонансного захвата;  
$\theta$ - коэффициент использования тепловых нейтронов (доля тепловых нейтронов, захваченных в топливе);  
$\eta$ - число эффективных нейтронов деления (быстрых нейтронов) на один тепловой нейтрон, захваченный в топливе.

Распишите в ячейке markdown с использованием latex-нотации ниже в общем виде формулы расчета каждого из сомножителей (см. lab_05-theory02):  
1. приведите формулу расчета объемной концентрации $N$ c использованием плотности вещества $\rho$, молярной массы $M$.  
2. распишите формулы для сомножителей, используя только обогащение урана $x$, микросечения $\sigma$, эффективный интеграл $I_{эф}$, объемные концентрации $N$, средний логарифмический декремент энергии $\xi$, объемные доли компонент $\epsilon$ и индексы, соответствующие типам компонент (Т, ТН, К, З)

Внимание! В работе принять $I_{эф} = 280б$. Расписывать формулу вычисления $I_{эф}$ не нужно.

Реализуйте расчет коэффициента размножения нейтронов на основе полученных формул в форме класса.  
Заготовка  представлена ниже.

In [3]:
class HomoReactor:
    'Ядерный реактор на тепловых нейтронах с бесконечной гомогенной активной зоной' 
    s_f_235 =  0 # микросечение деления U-235
    s_c_235 =  0 # микросечение захвата U-235
    s_s_235 =  0 # микросечение рассеяния U-235
    s_c_238 =  0 # микросечение захвата U-238
    s_s_238 =  0 # микросечение рассеяния U-238
    nu_f =     2.42 # число нейтронов деления
    mu =       0 # коэффициент размножения на быстрых нейтронах в гомогенной среде

    def __init__(self, components: Dict[str, str], t_props: pd.Series, other_props: pd.DataFrame):
        '''
        Аргументы:  
        - components (Dict[str, str])): ключ - тип компоненты (Т, ТН, К, З), значение - выбранное вещество  
        - t_props (pd.Series):          часть params, содержащая свойства (eps, N, x) выбранного топлива    
        - other_props (pd.DataFrame):   свойства ТН, К, З для выбранной тройки веществ и никаких других
        '''
        self.x = t_props['x'] # обогащение урана 
        self.t_eps = t_props['eps'] # относительный объем топлива в среде
        self.t_N = t_props['N'] # ядерная плотность топлива 

        self.tn_props = other_props.loc[components['ТН']]
        self.k_props = other_props.loc[components['К']]
        self.z_props = other_props.loc[components['З']]

    def set_x(self, x):
        'Установить обогащение топлива'
        pass

    def phi_calc(self):
        'Расчет вероятности избежать резонансного захвата'
        pass

    def theta_calc(self):
        'Расчет коэфф-та использования тепловых нейтронов'
        pass

    def eta_calc(self):
        'Расчет эффективного числа нейтронов деления'
        pass

    def k_calc(self):
        'Расчет коэффициента размножения'
        pass

## Подготовка к расчетам

### 1. Обогащение урана

С использованием библиотеки `mendeleev` получите доли U-235 и U-238 в природном уране и рассчитайте его обогащение. Внестите значение в переменную `x_natural` ниже.

In [18]:
# поиск необходимых данных и рассчет x_natural


x_natural = -1 

In [19]:
x_vals = [x_natural, *[i*0.01 for i in range(1, 6)]] # обогащение урана, используемое в работе
x_vals

[-1, 0.01, 0.02, 0.03, 0.04, 0.05]

### 2. Комбинации веществ в реакторе

Составьте все возможные комбинации веществ так, чтобы в составе реактора присутствовали все компоненты (по одному представителю Т, ТН, К, З)

In [61]:
# подсказка 1
print('Подсказка 1:', params.loc[params['type'] == 'К'].index)
# подсказка 2
print('Подсказка 2:', list(product(['x1','x2'], ['y1','y2'])))

Подсказка 1: Index(['Zr', 'Fe'], dtype='object')
Подсказка 2: [('x1', 'y1'), ('x1', 'y2'), ('x2', 'y1'), ('x2', 'y2')]


Выведите число комбинаций и сами комбинации

## Исследование

### 1. Расчет коэффициента размножения
Для всех значений обогащения урана $x$ рассчитайте коэффициент размножения нейтронов при каждой из комбинаций веществ в составе реактора с долями, соответствующими реакторам РБМК и ВВЭР.  

Отобразите полученные значения на графике. 

0. Приведите график в "читаемый" вид (подписи осей, грамотно выбранный масштаб, сетка (если уместна), легенда). 
1. На графике разделите подкритическую и надкритическую области (например, с использованием matplotlib.lines.Line2D).
2. Если цветов для различения полученных значений не хватает, используйте сочетание цвет + стиль маркера.

Приведите графики для долей РБМК и ВВЭР.


In [ ]:
# рбмк

In [ ]:
# ввэр

Вывод:

### 2. Зависимость сомножителей k от объемной доли компонент
Исследуйте поведение отдельных сомножителей коэффициента размножения при изменении долей материалов.  
1. Используйте двумерные графики, например, `plt.contourf` (см. синтаксис и примеры в [доках](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.contourf.html)), для перераспределения долей между двумя компонентами.  
2. Используйте одномерные графики, при изменении доли одного компонента на $\pm\Delta$ и пропорционального распределения $\mp\Delta$ между другими компонентами.

Вывод:

### 3. Для среды, состоящей только из урана и замедлителя, провести исследования п.1 и п.2.
Долю топлива принять равной доле топлива в РБМК/ВВЭР в зависимости от четности/нечетности первой буквы фамилии по порядку в алфавите соответственно.

In [56]:
name = 'К'
print('ВВЭР' if (ord(name) - ord('А') + 1) % 2 else 'РБМК')

ВВЭР


Вывод:

## Итоги
Привести в порядок ноутбук. Не должно быть пустых ячеек, черновых записей, ячеек, выводящих ошибки, некорректные графики и т.п. 
По окончании работы также удалить данную ячейку.